# TP1 - Aplicações de algoritmos geométricos

Esse projeto está em um repositório do Github e pode ser acessado pelo seguinte link:

[Kd-three-nearest-neigbour](https://github.com/giulianopenido/kd-three-nearest-neighbour)

No repositório estão salvos todos os datasets utilizados nos testes.

## Bibliotecas utilizadas

Utilizou-se a biblioteca numpy para realizar a leitura dos arquivos CSV e para executar operações vetoriais na base de dados.

In [103]:
import numpy as np

Utilizou-se a biblioteca heapq, que implementa a estrutura de dados heap de prioridade, que funciona analogamente à uma fila de prioridade, porém em formato de heap. A decisão de utilizar essa biblioteca ao invés de prover uma implementação própria dessa estrutura deve-se ao fato de ela existe na biblioteca STL de C++ e, portanto, poderia ser utilizada, de acordo com a especificação do trabalho.

In [104]:
import heapq

## Funções auxiliares

Função utilizada na leitura dos dados a partir de um arquivo CSV e na divisão do dataset em base de treino e base de testes. Observe que o caminho para o arquivo deve ser passado via parâmetro, assim como o percentual de divisão dos dados e a semente de aleatoriedade

In [105]:
def get_datasets_from_file(filepath, train_size = 0.7, seed = 42):
  dataset = np.unique(np.genfromtxt(filepath, delimiter = ','), axis=0)
  num_train_instances = int(len(dataset) * train_size)
  if(seed is not None):
    np.random.seed(seed)
  np.random.shuffle(dataset)
  train_set, test_set = dataset[:num_train_instances], dataset[num_train_instances:]
  return (train_set, test_set)

Função que computa a distância euclidiana entre dois pontos

In [106]:
def get_distance(p1, p2):
  return np.sqrt(np.sum(np.square(p1 - p2)))

## Kd Tree

Classe que representa um nó de árvore. Possui um atributo correspondente ao ponto cartesiano que armazena e um atributo para cada um de seus ramos (esquerdo e direito).

In [107]:
class Node:
  def __init__(self, left_node, right_node, point):
    self.left_node = left_node
    self.right_node = right_node
    self.point = point

  def is_leaf(self):
    return self.left_node == None and self.right_node == None

Classe correspondente à árvore KDTree, construída recursivamente. 

O processo de construção da árvore funciona da seguinte forma: O dataset é ordenado conforme a coordenada que será levada em conta no nível da árvore a ser construído. Depois de ordenado, ele é dividido no seu ponto de mediana e cria-se um nó com ele. Em sequência, executa-se o mesmo procedimento com cada uma das subpartições geradas da divisão, até que os datasets sejam indivisíveis(Um único ponto). Nessa situação, cria-se uma folha com o ponto restante. 

Observe que o Heapsort foi utilizado como método de ordenação na etapa de selecionar a mediana do dataset. Essa opção no projeto deve-se ao fato de que era necessário economizar memória auxiliar, dado que o algoritmo de construção da árvore demanda muita memória.



A classe KDTree também implementa o método de encontrar os K vizinhos mais próximos. Esse método funciona da seguinte forma:
A árvore é percorrida recursivamente, porém, apenas os ramos em que há chances de existir um vizinho próximo são acessados. À medida que percorre-se a árvore,  são selecionados pontos com base nas menores distâncias em relação ao ponto em análise. Os selecionados são armazenados em um Heap de prioridade (Estrutura de dados que prioriza os pontos cuja distância em relação ao ponto em análise está entre as menores). Ao final da execução, os pontos presentes no heap serão os K pontos mais próximos.

In [108]:
class Kd_tree:
  def __init__(self, dataset, ndim):
    self.ndim = ndim
    self.root = self.build_kd_tree(dataset)

  def build_kd_tree(self, dataset, depth=0):
    dataset_size = len(dataset)
    current_dimension = (depth) % self.ndim
    dataset = dataset[np.argsort(dataset[:, current_dimension], kind='heapsort')]
    if(dataset_size == 0):
      return None
    elif(dataset_size == 1):
      return Node(None, None, dataset[0])
    else:
      half = dataset_size // 2
      left = self.build_kd_tree(dataset[: half], depth+1)
      right = self.build_kd_tree(dataset[half:], depth+1)
      return Node(left, right, dataset[half]) 

  def recursive_k_nearest(self, k, current_node, point, priority_heap, depth=0):
    if(current_node == None):
      return

    current_dimension = depth % self.ndim
    distance = get_distance(point[:self.ndim], current_node.point[:self.ndim])

    if(len(priority_heap) < k and current_node.is_leaf()):
      try:
        heapq.heappush(priority_heap, (-distance, current_node.point))
      except ValueError: pass
    elif(current_node.is_leaf() and distance < abs(priority_heap[0][0])):
      try:
        heapq.heappushpop(priority_heap, (-distance, current_node.point))
      except ValueError: pass
    next_branch, other_branch = None, None

    if(point[current_dimension] < current_node.point[current_dimension]):
      next_branch, other_branch = current_node.left_node, current_node.right_node
    else: 
      next_branch, other_branch = current_node.right_node, current_node.left_node

    self.recursive_k_nearest(k, next_branch, point, priority_heap, depth+1)
    line_shortest_distance = abs(point[current_dimension] - current_node.point[current_dimension])
    if(abs(priority_heap[0][0]) > line_shortest_distance or len(priority_heap) < k ):
      self.recursive_k_nearest(k, other_branch, point, priority_heap, depth+1)

  def get_k_nearest(self, k, point):
    heap = []
    self.recursive_k_nearest(k, self.root, point, heap)
    return heap

  def get_three_structure(self, current_node, depth = 0):
    print(current_node.point, depth)
    if(current_node == None):
      return
    if(current_node.right_node == None and current_node.left_node == None):
      print(f'{depth}: {current_node.point}')
    else:
      self.get_three_structure(current_node.left_node, depth+1)
      self.get_three_structure(current_node.right_node, depth+1)

  def print_three(self):
    self.get_three_structure(self.root)

## KNN Classifier

A classe K_neighbours_classifier é uma abstração do uso da Kd Tree na busca pelos vizinhos próximos, tendo em vista que ela simplifica o uso do algoritmo por meio de funções simples e que são comuns a qualquer modelo de classificação. 
Observe que a função fit, que constrói a árvore, recebe como parâmetro o número de atributos do dataset. Esse valor é necessário para realizar a divisão dos dados entre atributos e classes.
A função predict, por sua vez, recebe o conjunto a partir do qual será realizada uma previsão e retorna as classes para as quais os pontos foram classificados.

In [109]:
class K_neighbours_classifier:
  def __init__(self, k = 2):
    self.k = k
  
  def fit(self, dataset, num_of_features):
    self.kd_tree = Kd_tree(dataset, num_of_features)

  def predict(self, points):
    predictions = []
    for point in points:
      knn_heap = self.kd_tree.get_k_nearest(self.k, np.array(point))
      frequencies = {}
      for near_point_data in knn_heap:
        distance, near_point = near_point_data
        near_point_class = near_point[-1]
        if near_point_class in frequencies:
            frequencies[near_point_class] += 1
        else:
            frequencies[near_point_class] = 1
      classes_ordered_by_frequency = sorted(frequencies.items(), key = lambda x: x[1], reverse=True)
      predictions.append(classes_ordered_by_frequency[0][0])
    return np.array(predictions)
    

  

A classe Metrics engloba algumas funções comuns de avaliação de modelos de classificação. Para o seu uso, é necessário instanciá-la passando no construtor os rótulos do dataset de teste e as predições do modelo.

In [110]:
class Metrics:
  def __init__(self, test_labels, predictions):
    self.test_labels = test_labels
    self.predictions = predictions

  def get_most_frequent_category(self):
    frequencies = {}
    for prediction in self.predictions:
      if prediction in frequencies:
        frequencies[prediction] += 1
      else:
        frequencies[prediction] = 1
    return sorted(frequencies.items(), key = lambda x: x[1], reverse=True)[0][0]

  def get_accuracy(self):
    correct_predictions = 0
    for value, prediction in zip(self.test_labels, self.predictions):
      if(value == prediction):
        correct_predictions += 1

    return correct_predictions/len(self.predictions)

  def get_precision(self):
    most_frequent_cat = self.get_most_frequent_category()
    true_positives = 0
    false_positives = 0
    for value, prediction in zip(self.test_labels, self.predictions):
      if(value == prediction and prediction == most_frequent_cat):
        true_positives += 1
      elif(value != most_frequent_cat and prediction == most_frequent_cat):
        false_positives += 1

    return true_positives/(true_positives + false_positives)

  def get_recall(self):
    most_frequent_cat = self.get_most_frequent_category()
    true_positives = 0
    false_negatives = 0
    for value, prediction in zip(self.test_labels, self.predictions):
      if(value == prediction and prediction == most_frequent_cat):
        true_positives += 1
      elif(value == most_frequent_cat and prediction != most_frequent_cat):
        false_negatives += 1

    return true_positives/(true_positives + false_negatives)

  def get_statistics(self):
    print(f"""
    Acurácia: { self.get_accuracy() }
    Precisão: { self.get_precision() }
    Revocação: { self.get_recall() }
    """)

## Casos de uso do modelo

Para testar os algoritmos e estruturas de dados implementados, utilizou-se 10 datasets diferentes e as estatísticas das previsões do modelo foram computadas, conforme consta abaixo:

In [114]:
# List of datasets --> Tuple of (path, num_of_features) 
datasets = [
  ('datasets/banana.csv', 2),
  ('datasets/appendicitis.csv', 7),
  ('datasets/bupa.csv', 6),
  ('datasets/haberman.csv', 3),
  ('datasets/monk-2.csv', 6),
  ('datasets/phoneme.csv', 5),
  ('datasets/pima.csv', 8),
  ('datasets/saheart.csv', 9),
  ('datasets/magic.csv', 10),
]

In [115]:
for dataset, num_of_features in datasets:
  print(f"-------- DATASET : {dataset} ----------")
  train_set, test_set = get_datasets_from_file(dataset)
  knn_classifier = K_neighbours_classifier(7)
  knn_classifier.fit(train_set, num_of_features)
  predictions = knn_classifier.predict(test_set)
  print("Avaliação do modelo: ")
  Metrics(test_set[:, -1], predictions).get_statistics()
  print("\n")

-------- DATASET : datasets/banana.csv ----------
Avaliação do modelo: 

    Acurácia: 0.896095717884131
    Precisão: 0.8982494529540481
    Revocação: 0.9193729003359462
    


-------- DATASET : datasets/appendicitis.csv ----------
Avaliação do modelo: 

    Acurácia: 0.9393939393939394
    Precisão: 0.967741935483871
    Revocação: 0.967741935483871
    


-------- DATASET : datasets/bupa.csv ----------
Avaliação do modelo: 

    Acurácia: 0.6310679611650486
    Precisão: 0.6282051282051282
    Revocação: 0.8596491228070176
    


-------- DATASET : datasets/haberman.csv ----------
Avaliação do modelo: 

    Acurácia: 0.7325581395348837
    Precisão: 0.75
    Revocação: 0.9344262295081968
    


-------- DATASET : datasets/monk-2.csv ----------
Avaliação do modelo: 

    Acurácia: 0.8076923076923077
    Precisão: 0.7402597402597403
    Revocação: 0.9193548387096774
    


-------- DATASET : datasets/phoneme.csv ----------
Avaliação do modelo: 

    Acurácia: 0.863013698630137
    P